In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_parquet('../data/yellow_tripdata_2023-01.parquet')

In [3]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [4]:
df.duration.std()

42.594351241920904

In [5]:
df = df[(df.duration >= 1) & (df.duration <= 60)]

In [6]:
len(df)*100/3066766

98.1220282212598

In [7]:
categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)

In [8]:
train_dicts = df[categorical].to_dict(orient='records')

In [9]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [10]:
X_train.shape[1]

515

In [ ]:
target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

In [15]:
y_pred = lr.predict(X_train)

In [16]:
rmse = np.sqrt(mean_squared_error(y_train, y_pred))

In [17]:
rmse

7.6492624397080675

In [18]:
df_val = pd.read_parquet('../data/yellow_tripdata_2023-02.parquet')

In [19]:
df_val.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1.0,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142.0,163.0,2.0,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00
1,2.0,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71.0,71.0,4.0,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00
2,2.0,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71.0,71.0,4.0,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00
3,1.0,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132.0,26.0,1.0,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25
4,2.0,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161.0,145.0,1.0,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00


In [20]:
df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)
df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]

In [21]:
df_val[categorical] = df_val[categorical].astype(str)

In [22]:
val_dicts = df_val[categorical].to_dict(orient='records')

In [28]:
X_val = dv.transform(val_dicts)

In [29]:
target = 'duration'
y_val = df_val[target].values

y_pred_val = lr.predict(X_val)
rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))

In [30]:
rmse_val

13.328465316947057